# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [4]:
data = pd.io.stata.read_stata('us_job_market_discrimination.dta')

In [5]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [6]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

# Question 1
- An A/B test to assess whether or not there is a significant difference between the mean callback rate from a resume with a white or black sounding name.
- CLT would require *normality, independence, and random* conditions.
    - The question states that the resumes were distributed **randomly**
    - This is a relatively large sample and we can assume normality because the number of successes and failures are both >= 10 for both the white sounding and black sounding name groups.
    - We can also assume independence because these ~5000 resumes constitute < 10% of all the resumes distributed to employers.
    
# Question 2
**Null Hypothesis**: There *is no* difference in callbacks between resumes with white sounding names and those with black sounding names.

**Alternate Hypothesis**: There *is a* difference.

In [26]:
w = data[data.race=='w']
b = data[data.race=='b']
w_call = w.call
b_call = b.call
len(w_call)

2435

In [21]:
# Your solution to Q3 here
np.random.seed(42)

def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates

def frac_call_b(b_call, w_call):
    """Compute fraction of black callbacks."""
    
    frac = np.sum(b_call) / len(b_call)
    return frac

# Acquire permutation samples: perm_replicates
perm_replicates = draw_perm_reps(b_call, w_call, frac_call_b, 10000)

# Compute and print p-value: p
p = np.sum(perm_replicates <= 157/2435) / len(perm_replicates)
print('p-value =', p)

p-value = 0.0


Frequentist
![Frequentist](Margin of Error 2.jpg)

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

# Question 4
Therefore, we can see from the 95% confidence intervals that there is no overlap in the mean callback rates between resumes with black or white sounding names. We can also see that from 10000 bootstrap samples assuming the null hypothesis, there were no means as extreme as the observed ratio of 153/2435 (resumes with black sounding names and callbacks).

# Question 5
We can see that there is a significant difference in the callback ratio between resumes that have white sounding names and black sounding names. This will lead us to conclude that within the 95% confidence intervals we are confident that the callback rates are between 1-5% higher for resumes with white sounding names.